<a href="https://colab.research.google.com/github/macvieira97/storytelling-with-data/blob/main/storytelling_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2: Choosing an effective visual

In [1]:
pip install chart_studio

In [2]:
import chart_studio as cs
cs.tools.set_credentials_file(username='macv97', api_key='rW2ViBF5BhcWxasj9Ijb')

In [3]:
import plotly.graph_objects as go
import chart_studio.plotly as py
import pandas as pd
from pandas import DataFrame

# Exercise 2.1

In [206]:
dfs = pd.read_excel('data1.xlsx')
dfs

,EXERCISE 2.1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,FIG 2.1a,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,New client tier share,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
6,NaN,A,77,0.070772,4.675,0.25
7,NaN,A+,19,0.017463,3.927,0.21
8,NaN,B,338,0.310662,5.984,0.32
9,NaN,C,425,0.390625,2.805,0.15


In [207]:
for i in range(4):
  column_name = "Unnamed: "+str(i+1)
  dfs = dfs[dfs[column_name].notna()]
dfs = dfs.drop(columns=["EXERCISE 2.1"])
list = dfs.iloc[0]
keys = []
values = []
for name in dfs.columns:
  keys.append(name)
  values.append(list[name])

dict_columns = dict(zip(keys, values))
dfs = dfs.rename(columns=dict_columns)
dfs = dfs.reset_index(drop=True)
dfs = dfs.drop(axis=0, index=0)
dfs

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
1,A,77,0.070772,4.675,0.25
2,A+,19,0.017463,3.927,0.21
3,B,338,0.310662,5.984,0.32
4,C,425,0.390625,2.805,0.15
5,D,24,0.022059,0.374,0.02


In [208]:
for i in range(len(dfs.Tier)):
  dfs.iloc[i, 2] = dfs.iloc[i, 2]*100
  dfs.iloc[i, 4] *= 100
dfs

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
1,A,77,7.077206,4.675,25.0
2,A+,19,1.746324,3.927,21.0
3,B,338,31.066176,5.984,32.0
4,C,425,39.0625,2.805,15.0
5,D,24,2.205882,0.374,2.0


In [209]:
col_names = []
for i in values:
  col_names.append([i])
col_names

[['Tier'], ['# of Accounts'], ['% Accounts'], ['Revenue ($M)'], ['% Revenue']]

In [210]:
dfs = dfs.reindex([2, 1, 3, 4, 5]).reset_index(drop=True)


In [211]:
sum = dfs.sum().to_frame().transpose()
per_account = sum['% Accounts'].apply(lambda x: 100-x)
dif_account = per_account * sum['# of Accounts']/sum['% Accounts']
per_revenue = sum['% Revenue'].apply(lambda x: 100-x)
dif_revenue = per_revenue * sum['Revenue ($M)']/sum['% Revenue']
difference = {'Tier': 'All others', '# of Accounts': dif_account, '% Accounts': per_account, 'Revenue ($M)': dif_revenue, '% Revenue': per_revenue}

dif = pd.DataFrame(difference)
total = pd.concat([dfs, dif])

total_sum = total.sum().to_dict()
total_sum['Tier']=['TOTAL']
total_sum = pd.DataFrame(total_sum)

#total_sum = total_sum.to_frame().transpose()
total = pd.concat([dfs, dif, total_sum])
total

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
0,A+,19,1.746324,3.927,21.0
1,A,77,7.077206,4.675,25.0
2,B,338,31.066176,5.984,32.0
3,C,425,39.0625,2.805,15.0
4,D,24,2.205882,0.374,2.0
0,All others,205.0,18.841912,0.935,5.0
0,TOTAL,1088.0,100.0,18.7,100.0


In [212]:
total['Rank_account'] = total['% Accounts'].rank()
total['Rank_account'] = total['Rank_account'].astype(int)
total['Rank_revenue'] = total['% Revenue'].rank()
total['Rank_revenue'] = total['Rank_revenue'].astype(int)
total

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue,Rank_account,Rank_revenue
0,A+,19,1.746324,3.927,21.0,1,4
1,A,77,7.077206,4.675,25.0,3,5
2,B,338,31.066176,5.984,32.0,5,6
3,C,425,39.0625,2.805,15.0,6,3
4,D,24,2.205882,0.374,2.0,2,1
0,All others,205.0,18.841912,0.935,5.0,4,2
0,TOTAL,1088.0,100.0,18.7,100.0,7,7


In [213]:
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

colors = n_colors('rgb(208, 228, 197)', 'rgb(114, 172, 77)', 10, colortype='rgb')
total['Rank_account'] = total['% Accounts'].rank() - 1
total['Rank_account'] = total['Rank_account'].astype(int)
total['Rank_revenue'] = total['% Revenue'].rank() - 1
total['Rank_revenue'] = total['Rank_revenue'].astype(int)


table_colored = go.Figure(data=[go.Table(
    header=dict(values=col_names,
                fill_color='#F2F2F2',
                line_color='#F1F1F1',
                align='center'),
    cells=dict(values=[total['Tier'], total['# of Accounts'].map("{:,.0f}".format),
                       total['% Accounts'].map("{:,.0f}%".format),
                       total['Revenue ($M)'].map("{:,.1f}".format),
                       total['% Revenue'].map("{:,.0f}%".format)],
               fill_color=['white', 'white', np.array(colors)[total['Rank_account']], 'white', np.array(colors)[total['Rank_revenue']]],
               line_color='#F1F1F1',
               align='center', font=dict(color='black', size=11)))],
    layout=dict(title=dict(text="New client tier share"), title_font=dict(size=20))
)
table_colored.layout.update(margin=dict(l=50, r=10, b=10, t=100, pad=1))

table_colored.layout.update({'width' : 700})

table_colored.show()

In [214]:
p_total = pd.concat([dfs, dif])
p_total

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
0,A+,19,1.746324,3.927,21.0
1,A,77,7.077206,4.675,25.0
2,B,338,31.066176,5.984,32.0
3,C,425,39.0625,2.805,15.0
4,D,24,2.205882,0.374,2.0
0,All others,205.0,18.841912,0.935,5.0


In [269]:
import plotly.subplots as subplot

fig = subplot.make_subplots(rows=1, cols=2, horizontal_spacing=0.05)

accounts_bar = fig.add_bar(
    x=p_total['% Accounts'][::-1],
    y=p_total['Tier'][::-1],
    orientation='h',
    text=p_total['% Accounts'][::-1].map("{:,.0f}%".format),
    hoverinfo='text',
    width=0.6,
    marker=dict(color='rgba(155, 190, 250, 0.7)'),
    showlegend=False,
    row=1, col=1)

revenue_bar = fig.add_bar(
    x=p_total['% Revenue'][::-1],
    y=p_total['Tier'][::-1],
    orientation='h',
    text=p_total['% Revenue'][::-1].map("{:,.0f}%".format),
    hoverinfo='text',
    marker=dict(color='rgba(100, 159, 237, 110)'),
    width=0.6,
    showlegend=False,
    row=1, col=2)

fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=1, col=2)

fig.update_traces(textposition='inside', textfont_size=12)

fig.update_layout(
    title_text='New client tier share',
    title_x=0,  # Set the title's x-position to center
    height=500,
    width=1000,
    paper_bgcolor="white",
    plot_bgcolor="white",
     annotations=[
        dict(
            text='Tier | % OF TOTAL <span style="color:rgba(155, 190, 250, 0.7);">ACCOUNTS</span> | % OF TOTAL <span style="color:rgba(100, 159, 237, 110);">REVENUE</span>',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=-0.01,  # Set the x-position of the subtitle
            y=1.1,  # Set the y-position of the subtitle
            font=dict(size=13)
        )
    ],
    autosize=True
)

fig.show()

In [204]:
from plotly.subplots import make_subplots
# Create subplots with two bar charts
fig = make_subplots(rows=1, cols=2)
colors_1 = n_colors('rgb(100, 149, 237)', 'rgb(100, 129, 257)', 7, colortype='rgb')
colors_2 = n_colors('rgb(135, 206, 250)', 'rgb(135, 206, 235)', 7, colortype='rgb')

fig.add_annotation(
    #text='% of Accounts and Revenue',
    text='% OF TOTAL <span style="color:rgba(155, 190, 250, 0.7);">ACCOUNTS</span> vs. <span style="color:rgba(100, 159, 237, 1);">REVENUE</span>',
    xref='paper', yref='paper',
    x=-0.02, y=1.2,  # Adjust the x and y values to position the subtitle
    showarrow=False,
    font=dict(size=12, color='black')
)
fig.add_trace(go.Bar(
    x=p_total['% Revenue'],
    y=p_total['Tier'],
    orientation='h',
    showlegend=False,
    marker=dict(color='rgba(100, 159, 237, 1)'),
    name="% Revenue"),
)
fig.add_trace(go.Bar(
    x=p_total['% Accounts'],
    y=p_total['Tier'],
    orientation='h',
    showlegend=False,
    marker=dict(color='rgba(155, 190, 250, 0.7)'),
    name="% Accounts"),
)

# Update layout
#Title
fig.update_layout(title_text='New client tier share')
#Background grid
fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)')
# Tick notation
fig.update_layout(# on the y-axis
                  xaxis_ticksuffix="%",
                  # on the colorbar
                  coloraxis_colorbar_ticksuffix="m",
                  # To specify which tick should have suffix
                  xaxis_showticksuffix="all",  # or "first" or "last"
                  xaxis={'side': 'top'},
                  #yaxis={'side': 'right'}
                  yaxis={'categoryorder':'array', 'categoryarray':['All others', 'D', 'C', 'B', 'A', 'A+']})
fig.layout.update(margin=dict(l=50, r=10, b=50, t=150, pad=10))
fig.layout.update({'width' : 1000})

fig.update_xaxes(ticks="outside", tickwidth=1, tickcolor='rgba(169, 169, 169, 1)', ticklen=10)
fig.update_xaxes(showline=True, linewidth=1, linecolor='rgba(169, 169, 169, 1)')


# Show the figure
fig.show()


In [205]:
from plotly.subplots import make_subplots
# Create subplots with two bar charts
fig = make_subplots(rows=1, cols=2)
#colors_1 = n_colors('rgb(100, 149, 237)', 'rgb(100, 129, 257)', 7, colortype='rgb')
#colors_2 = n_colors('rgb(135, 206, 250)', 'rgb(135, 206, 235)', 7, colortype='rgb')


fig.add_annotation(
    #text='% of Accounts and Revenue',
    text='% OF TOTAL <span style="color:rgba(155, 190, 250, 0.7);">ACCOUNTS</span> vs. <span style="color:rgba(100, 159, 237, 1);">REVENUE</span>',
    xref='paper', yref='paper',
    x=-0.04, y=1.1,  # Adjust the x and y values to position the subtitle
    showarrow=False,
    font=dict(size=12, color='black'))

# Add the first bar chart to the first subplot
fig.add_trace(go.Bar(
    x=p_total['Tier'],
    y=p_total['% Accounts'],
    showlegend=False,
    marker=dict(color='rgba(155, 190, 250, 0.7)'),
    name="% Accounts"),
)

# Add the second bar chart to the second subplot
fig.add_trace(go.Bar(
    x=p_total['Tier'],
    y=p_total['% Revenue'],
    showlegend=False,
    marker=dict(color='rgba(100, 159, 237, 1)'),
    name="% Revenue"),
)

# Update layout
fig.update_layout(title_text='New client tier share',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  # on the y-axis
                  yaxis_ticksuffix="%",
                  # on the colorbar
                  coloraxis_colorbar_ticksuffix="m",
                  # To specify which tick should have suffix
                  yaxis_showticksuffix="all"  # or "first" or "last"
                  )

fig.update_yaxes(ticks="outside", tickwidth=1, tickcolor='rgba(169, 169, 169, 1)', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='rgba(169, 169, 169, 1)')
fig.layout.update({'width' : 1000})

# Show the figure
fig.show()

In [21]:
p_total

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
0,A+,19,1.746324,3.927,21.0
1,A,77,7.077206,4.675,25.0
2,B,338,31.066176,5.984,32.0
3,C,425,39.0625,2.805,15.0
4,D,24,2.205882,0.374,2.0
0,All others,205.0,18.841912,0.935,5.0


In [200]:
fig = go.Figure()
values=[]
for index, row in p_total.iterrows():
  fig.add_trace(go.Scatter(x=[0, 0.01],
                  y=[row['% Accounts'], row['% Revenue']],
                  mode='lines+markers+text',
                  text=["{:,.0f}%".format(row['% Accounts']), "{:,.0f}%".format(row['% Revenue'])],
                  textposition=['middle left', 'middle right'],
                  line=dict(color='gray', width=4),
                  marker=dict(color='gray', size=12),
                  showlegend=False))
  values.append("{:,.0f}%".format(row['% Revenue']))

#fig.add_shape(type='line', x0=0, x1=0, y0=0, y1=1, xref='x', yref='paper')
#fig.add_shape(type='line', x0=1, x1=1, y0=0, y1=1, xref='x', yref='paper')

# Add text annotations

fig.add_annotation(
    text='% of TOTAL <br><b>REVENUE</b></br>',
    xref='paper', yref='paper',
    x=1.01, y=-0.1,  # Adjust the x and y values to position the subtitle
    showarrow=False,
    font=dict(size=15, color='black'))

fig.add_annotation(
    text='% of TOTAL <br><b>ACCOUNTS</b></br>',
    xref='paper', yref='paper',
    x=0, y=-0.1,  # Adjust the x and y values to position the subtitle
    showarrow=False,
    font=dict(size=15, color='black'))

# Update layout
fig.update_layout(title_text='New client tier share', font=dict(size=13, color='black'))
fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)')
fig.update_layout(xaxis=dict(showticklabels=False),
                  yaxis=dict(tickvals=values, ticktext=p_total['Tier'], side='right')
                  )
fig.layout.update(margin=dict(l=20, r=20, b=50, t=100, pad=10))
fig.layout.update({'width': 725, 'height': 500})

fig.show()